In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

indir = '/Users/TommyFellowes/OneDrive - The University of Sydney (Staff)/2 Supervising/2022_Roncolato_Fran/Matlab_Scripts_RBR'
outdir = '/Users/TommyFellowes/OneDrive - The University of Sydney (Staff)/2 Supervising/2022_Roncolato_Fran/RBR_Test_Data_RoseBay/Output'

# Date check
data_raw = np.genfromtxt(indir + '/data_raw.csv', delimiter=',')

plt.figure()
plt.plot(data_raw[:, 0], data_raw[:, 1])
plt.gcf().autofmt_xdate()
plt.show()

# Truncate Data (optional)
st = np.where(data_raw[:, 0] == datetime.datetime(2023, 7, 2, 14, 0))[0][0]  # T0 start data point
en = np.where(data_raw[:, 0] == datetime.datetime(2023, 7, 3, 12, 0))[0][0]  # T1 end data point

# truncate data
data_raw_tr = data_raw[st:en+1, :]

plt.plot(data_raw[:, 0], data_raw[:, 1])
plt.plot(data_raw_tr[:, 0], data_raw_tr[:, 1])
plt.gcf().autofmt_xdate()
plt.show()

# replay original with truncated data
data_raw = data_raw_tr
del data_raw_tr

# Reshape data for spectral analyses
Hz = 2  # sample frequency
#wave_info = data_raw[:, 2]
rl = 30  # min
dt = 1 / Hz  # sec
bl = int((rl / dt) * 60)  # burst length
nbl = int(np.floor(data_raw.shape[0] / bl))

data_time = np.reshape(data_raw[:nbl*bl, 0], (bl, nbl))
data_press = np.reshape(data_raw[:nbl*bl, 1], (bl, nbl))
data_segment = rl  # length of segments after reshaping data

# Mean water level for each run
wave_mpress = np.mean(data_press, axis=0)

# Gravity (local wind and swell) spectral wave parameters (1 to 20 s)
Gwave_time = np.empty(nbl)
Gwave_Hm0 = np.empty(nbl)
Gwave_Hrms = np.empty(nbl)
Gwave_Hmax = np.empty(nbl)
Gwave_Tm02 = np.empty(nbl)
Gwave_Tm01 = np.empty(nbl)
Gwave_Tpeak = np.empty(nbl)
Gwave_eps = np.empty(nbl)
Gwave_s = np.empty((nbl, bl))
Gwave_fspec = np.empty((nbl, bl))

for k in range(nbl):
    Gwave_time[k] = data_time[0, k]
    p = f_PT_power(data_press[:, k], dt, 1)

    out = f_Gwaves_spectral_RBR(p.px, p.f, dt)

    Gwave_time[k] = data_time[0, k]
    Gwave_Hm0[k] = out.hs
    Gwave_Hrms[k] = out.hrms
    Gwave_Hmax[k] = out.hmax
    Gwave_Tm02[k] = out.tm02
    Gwave_Tm01[k] = out.tm01
    Gwave_Tpeak[k] = out.tpeak
    Gwave_eps[k] = out.eps
    Gwave_s[k, :] = p.px  # spectrum
    Gwave_fspec[k, :] = p.f

# Infragravity spectral wave parameters
IGwave_time = np.empty(nbl)
IGwave_Hm0 = np.empty(nbl)
IGwave_Hrms = np.empty(nbl)
IGwave_Hmax = np.empty(nbl)
IGwave_Tm02 = np.empty(nbl)
IGwave_Tm01 = np.empty(nbl)
IGwave_eps = np.empty(nbl)
IGwave_s = np.empty((nbl, bl))
IGwave_fspec = np.empty((nbl, bl))

for k in range(nbl):
    IGwave_time[k] = data_time[0, k]
    p = f_PT_power(data_press[:, k], dt, 1)

    out = f_IGwaves_spectral_RBR(p.px, p.f, dt)

    IGwave_time[k] = data_time[0, k]
    IGwave_Hm0[k] = out.hs
    IGwave_Hrms[k] = out.hrms
    IGwave_Hmax[k] = out.hmax
    IGwave_Tm02[k] = out.tm02
    IGwave_Tm01[k] = out.tm01
    IGwave_eps[k] = out.eps
    IGwave_s[k, :] = p.px
    IGwave_fspec[k, :] = p.f

# Plots
[f2, f3, f4] = f_PT_plot_spectral_NEW(wave_info, Gwave_info, IGwave_info)

# Save figures
plt.savefig(outdir + '/Spectral_ww.png')
plt.savefig(outdir + '/Spectral_ss.png')
plt.savefig(outdir + '/Spectral_IG.png')
plt.savefig(outdir + '/Spectral_fspec.png')
plt.savefig(outdir + '/Zero_up.png')

# Save data
np.savetxt(outdir + '/wave_processed.csv', wave_info, delimiter=',')
np.savetxt(outdir + '/Gwave_processed.csv', Gwave_info, delimiter=',')
np.savetxt(outdir + '/IGwave_processed.csv', IGwave_info, delimiter=',')
np.savetxt(outdir + '/zeroup_processed.csv', zeroup_info, delimiter=',')